In [1]:
# The code was removed by Watson Studio for sharing.

Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = app-20190906073242-0001
KERNEL_ID = c41fa2d3-7578-4c35-91e0-ec025ee323e5


In [2]:
df.createOrReplaceTempView('df')
df_two_class = spark.sql("select * from df where class in ('Use_telephone','Standup_chair')")

In [3]:
splits = df_two_class.randomSplit([0.8, 0.2])
df_train = splits[0]
df_test = splits[1]

In [4]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import Normalizer


indexer = StringIndexer(inputCol="class", outputCol="label")
vectorAssembler = VectorAssembler(inputCols=["x","y","z"],outputCol="features")
normalizer = Normalizer(inputCol="features", outputCol="features_norm", p=1.0)

In [5]:
from pyspark.ml.classification import GBTClassifier
gbt = GBTClassifier(labelCol="label", featuresCol="features", maxIter=10)

In [6]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[indexer, vectorAssembler, normalizer,gbt])

In [8]:
%%time
model = pipeline.fit(df_train)

CPU times: user 108 ms, sys: 52 ms, total: 160 ms
Wall time: 25.3 s


In [9]:
prediction = model.transform(df_train)

In [10]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
binEval = MulticlassClassificationEvaluator().setMetricName("accuracy") .setPredictionCol("prediction").setLabelCol("label")
binEval.evaluate(prediction) 

0.9107966684212155

In [11]:
prediction = model.transform(df_test)

In [12]:
binEval.evaluate(prediction) 

0.9085680047932894

In [13]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

paramGrid = ParamGridBuilder() \
    .addGrid(normalizer.p, [1.0, 2.0, 10.0]) \
    .addGrid(gbt.maxBins, [2,4,8,16]) \
    .addGrid(gbt.maxDepth, [2,4,8,16]) \
    .build()

In [14]:
crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=MulticlassClassificationEvaluator(),
                          numFolds=4)  


In [ ]:
%%timeit
cvModel = crossval.fit(df_train)

In [33]:
prediction = cvModel.transform(df_test)

In [34]:
binEval.evaluate(prediction) 

0.9121697059918125

In [43]:
cvModel.bestModel.stages[-1].explainParams()

"cacheNodeIds: If false, the algorithm will pass trees to executors to match instances with nodes. If true, the algorithm will cache node IDs for each instance. Caching can speed up training of deeper trees. (default: False)\ncheckpointInterval: set checkpoint interval (>= 1) or disable checkpoint (-1). E.g. 10 means that the cache will get checkpointed every 10 iterations. Note: this setting will be ignored if the checkpoint directory is not set in the SparkContext (default: 10)\nfeatureSubsetStrategy: The number of features to consider for splits at each tree node. Supported options: auto, all, onethird, sqrt, log2, (0.0-1.0], [1-n]. (undefined)\nfeaturesCol: features column name (default: features, current: features)\nimpurity: Criterion used for information gain calculation (case-insensitive). Supported options: entropy, gini (undefined)\nlabelCol: label column name (default: label, current: label)\nlossType: Loss function which GBT tries to minimize (case-insensitive). Supported o